<a href="https://colab.research.google.com/github/june-oh/cseg109/blob/main/Lab2_LM_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

text = ("태양계는 매우 다양하고 흥미로운 천체들로 가득 차 있습니다. 이곳에는 여덟 개의 큰 행성과 다수의 작은 행성들이 있으며, 각각 독특한 특징을 지니고 있습니다. "
        "예를 들어, 수성은 태양에 가장 가까운 행성으로, 매우 뜨거운 낮과 매우 추운 밤을 경험합니다. 반면에, 목성은 가장 큰 행성으로 강력한 자기장을 가지고 있습니다. "
        "지구는 이 태양계에서 유일하게 생명이 존재하는 것으로 알려진 행성입니다. 물, 산소, 그리고 적당한 온도 덕분에 다양한 생물들이 이곳에서 살아가고 있습니다. "
        "인간을 포함한 많은 생명체들이 지구의 자연 환경에 적응하여 살아가고 있죠. 과학자들은 지구 밖의 다른 행성들에서도 생명의 흔적을 찾기 위해 끊임없이 탐사를 계속하고 있습니다. "
        "우주 탐사선과 망원경을 이용하여 멀리 떨어진 행성들과 별들을 관측하고 있죠. 이러한 연구는 우리가 우주에 대해 더 많이 알 수 있게 해주며, "
        "언젠가는 다른 행성에서도 생명을 찾을 수 있을 것이라는 희망을 줍니다. 우주는 아직도 많은 비밀을 간직하고 있습니다. 이를 탐구하는 과정은 인간의 호기심을 자극하고, "
        "과학적 지식의 발전에 기여하고 있습니다. 어쩌면 머지않은 미래에 우리는 우주 여행을 일상처럼 할 수 있게 될지도 모릅니다. 그때가 되면, 우주는 더 이상 먼 곳이 아니라 "
        "우리 생활의 일부가 될 것입니다.")


# 토큰화 및 어휘 사전 생성
tokens = text.split()
vocab = set(tokens)
word_to_idx = {word: i for i, word in enumerate(vocab)}

# 토큰을 인덱스로 변환
indexed_tokens = [word_to_idx[word] for word in tokens]

# 텐서로 변환
tensor_tokens = torch.tensor(indexed_tokens, dtype=torch.long)


UnigramLM은 앞의 한단어만을 보고 다음 단어를 예측하도록 작성

In [ ]:
class UnigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(UnigramLanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        #적절한 layer를 추가하세요.
        self.linear = nn.Linear(embedding_dim, vocab_size)


    def forward(self, inputs):
        #forward 함수를 작성하세요.
        embeds = self.embeddings(inputs)
        out = self.linear(embeds)
        return out

In [ ]:
# 모델 인스턴스 생성
vocab_size = len(vocab)
embedding_dim = 128  # 예시로 설정한 임베딩 차원
model = UnigramLanguageModel(vocab_size, embedding_dim)

# 학습 데이터 준비
# 각 토큰에 대해 다음 토큰을 예측하는 작업을 수행합니다.
# 각각을 작성하세요
train_data = tensor_tokens[:-1]
target_data = tensor_tokens[1:]


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for i in range(len(train_data)):
        model.zero_grad()
        output = model(train_data[i].unsqueeze(0))
        loss = loss_function(output, target_data[i].unsqueeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0:
      print(f'Epoch {epoch}: Total Loss {total_loss/len(train_data)}')

Epoch 0: Total Loss 5.0723517969250675
Epoch 10: Total Loss 0.41751954627688975
Epoch 20: Total Loss 0.3339752217754722
Epoch 30: Total Loss 0.3164331726700766
Epoch 40: Total Loss 0.3072407048282912


In [ ]:
def generate_sentence(model, start_token, word_to_idx, idx_to_word, max_length=20):
    """
    문장 생성 함수
    :param model: 학습된 모델
    :param start_token: 문장 생성을 시작할 첫 단어
    :param word_to_ix: 단어를 인덱스로 변환하는 사전
    :param ix_to_word: 인덱스를 단어로 변환하는 사전
    :param max_length: 생성할 문장의 최대 길이
    :return: 생성된 문장
    """
    current_word = start_token
    sentence = [current_word]
    model.eval()  # 모델을 추론 모드로 설정

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for _ in range(max_length - 1):
            #auto regressive 하게 입력이 바뀌도록 작성하세요.
            #mystery = 현재입력
            mystery =  word_to_idx[current_word]
            token_index = torch.tensor([mystery], dtype=torch.long)
            output = model(token_index)
            next_word_idx = output.argmax(1).item()
            next_word = idx_to_word[next_word_idx]

            sentence.append(next_word)
            current_word = next_word

    return ' '.join(sentence)

# 인덱스를 단어로 변환하는 사전 생성
idx_to_word = {i: word for word, i in word_to_idx.items()}


In [ ]:
print(vocab)

{'것으로', '천체들로', '태양계에서', '있습니다.', '지구는', '존재하는', '생명을', '연구는', '밤을', '우리가', '행성들에서도', '알', '강력한', '독특한', '이러한', '수', '위해', '있을', '다양하고', '될', '산소,', '알려진', '인간을', '물,', '발전에', '가까운', '다양한', '가지고', '행성으로,', '자극하고,', '큰', '적응하여', '먼', '탐사선과', '행성으로', '적당한', '행성들과', '생명이', '낮과', '이를', '지식의', '대해', '어쩌면', '아니라', '멀리', '작은', '끊임없이', '곳이', '아직도', '할', '기여하고', '일상처럼', '다수의', '탐구하는', '여행을', '언젠가는', '것이라는', '추운', '유일하게', '별들을', '우리는', '흥미로운', '과학적', '반면에,', '포함한', '흔적을', '많이', '특징을', '태양계는', '매우', '예를', '살아가고', '관측하고', '태양에', '환경에', '각각', '행성들이', '여덟', '행성과', '이곳에는', '되면,', '찾을', '우주는', '희망을', '자기장을', '것입니다.', '생명의', '인간의', '호기심을', '머지않은', '있으며,', '망원경을', '우주', '이상', '그때가', '자연', '많은', '생물들이', '이곳에서', '가장', '더', '차', '생활의', '줍니다.', '행성에서도', '그리고', '모릅니다.', '미래에', '들어,', '개의', '비밀을', '밖의', '지니고', '과정은', '이', '해주며,', '찾기', '지구의', '떨어진', '있게', '뜨거운', '탐사를', '일부가', '온도', '생명체들이', '계속하고', '목성은', '간직하고', '될지도', '행성입니다.', '다른', '우리', '경험합니다.', '수성은', '있죠.', '지구', '가득', '이용하여', '과학자들은', '덕분에', '우주에'}


In [ ]:
start_token = '강력한'  # 시작 단어 설정
generated_sentence = generate_sentence(model, start_token, word_to_idx, idx_to_word)
print(generated_sentence)

강력한 자기장을 가지고 있습니다. 어쩌면 머지않은 미래에 우리는 우주 여행을 일상처럼 할 수 있게 될지도 모릅니다. 그때가 되면, 우주는 더


In [ ]:
start_token = '인간의'  # 시작 단어 설정
generated_sentence = generate_sentence(model, start_token, word_to_idx, idx_to_word)
print(generated_sentence)

인간의 호기심을 자극하고, 과학적 지식의 발전에 기여하고 있습니다. 어쩌면 머지않은 미래에 우리는 우주 여행을 일상처럼 할 수 있게 될지도 모릅니다.
